# Linear regression

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lento234/ml-tutorials/blob/main/01-basics/linear_regression.ipynb)

**References**:
- https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/linear_regression/main.py
- https://en.wikipedia.org/wiki/Linear_regression

**Linear regression**

Given a data set $\{y_i, x_i\}_{i=1}^n$ of $n$ statistical units, we have the following relationship:

$$ y_i = \beta_1 x_i + \beta_0 + \varepsilon_i, \qquad\quad i=1,\ldots, n,$$

where:
- $y_i$ is observed value (dependent variable
- $x_i$ is the independent variable
- $\beta_1$ and $\beta_0$ are the trainable parameters
- $\varepsilon_i$ is the measurement error


### Setup

#### Load packages / modules

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.style.use('seaborn-poster')
mpl.rcParams['figure.figsize'] = [8, 5]

#### Generate training dataset

In [ ]:
n = 100
beta_1 = 2.0
beta_0 = 0.3
eps_std = 0.1

In [ ]:
x_train = np.random.rand(n,1)
eps = np.random.randn(n,1)*eps_std
y_train = x_train*beta_1 + beta_0 + eps

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_train, y_train, 'k.', label='observations')
ax.plot(x_train, x_train*beta_1 + beta_0, 'tab:gray',
        label=r'analytical ($\beta_1={:.2f}$, $\beta_0={:.2f}$)'.format(beta_1, beta_0))
ax.set(xlabel='x', ylabel='y', xlim=(0,1), ylim=(0.1, 2.4));
ax.legend();

### Setup model

#### Hyperparameters

In [ ]:
n_features = 1 # number of nodes (1 weight + 1 bias)
num_epochs = 100 # i.e. number of iterations
learning_rate = 0.2

#### Model, Loss and Optimizer

In [ ]:
# Linear regression model
model = nn.Linear(in_features=n_features, out_features=n_features)

# Loss and optimizer
criterion = nn.MSELoss() # Mean-Square Error loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # Stochastic gradient-descent

### Train the model

In [ ]:
loss_history = []
for epoch in range(num_epochs):
    # Convert numpy arrays to torch tensors
    x = torch.from_numpy(x_train.astype('float32'))
    y = torch.from_numpy(y_train.astype('float32'))

    # Forward pass
    y_pred = model(x) # prediction step
    loss = criterion(y_pred, y)
    
    # Backward and optimize
    optimizer.zero_grad() # reset gradient
    loss.backward() # backward propogation 
    optimizer.step() # update gradient
    
    loss_history.append(loss.item())
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
fig, ax = plt.subplots()
ax.plot(loss_history, '.-')
ax.set(xlabel='epoch', ylabel='MSE loss',
       title='Training loss history');

### Predict and evaluate

In [ ]:
# Predict
predict = model(torch.from_numpy(x_train.astype('float32'))).detach().numpy()
parameters = [param.detach().item() for param in model.parameters()]

# Plot the graph
fig, ax = plt.subplots()
ax.plot(x_train, y_train, 'k.', label='observations')
ax.plot(x_train, x_train*beta_1 + beta_0, 'tab:gray',
        label=r'analytical: ($\beta_1={:.2f}$, $\beta_0={:.2f}$)'.format(beta_1, beta_0))
ax.plot(x_train, predict, 'tab:red', 
        label=r'predicted: ($\beta_1={:.2f}$, $\beta_0={:.2f}$)'.format(*parameters))
ax.set(xlabel='x', ylabel='y',
       title='Training accuracy',
      xlim=(0,1), ylim=(0.1, 2.4));
ax.legend()

### Save the trained model

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

### Load pretrained model

In [ ]:
checkpoint = torch.load('model.ckpt')
checkpoint

In [ ]:
model.load_state_dict(checkpoint)